In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg
import numpy as np
import galois   
import random

In [2]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [3]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 125 100 1250


In [4]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
# print(L_vector_of_poly)

In [5]:
def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ux = reduce_polynomials_to_1(L_vector_of_poly,wgf)
Vx = reduce_polynomials_to_1(R_vector_of_poly,wgf)
Wx = reduce_polynomials_to_1(O_vector_of_poly,wgf)

Sample output from the above steps

7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186575808494842
13680151794899547013904003590785796930342727750260021464811377616609880309765x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539795x^3 + 8208091076939728208342402154471478158205636650156012878886826569965928186022x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955532x + 125
6384070837619788606488535009033371900826606283454676683578642887751277477581x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253543130x^3 + 15504172034219486615757870736223903187721758116961357660119561298824531005611x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554973812x + 21888242871839275222246405745257275088548364400416034343698204186575808486742

In [6]:
UxVx = Ux*Vx   
Tx = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])).astype(int),  field=GF)
Hx, remainder = divmod(((Ux*Vx) - Wx), Tx)
assert remainder == 0, "remainder is not 0"

# Check Tx and Hx balances the polynomial correctly
lhs = Ux*Vx
rhs = Hx*Tx + Wx
assert lhs == rhs, "result is not balanced across LHS and RHS"

Preventing forgery, introduce $\alpha$ and $\beta$

### Pairing LHS:
$(\alpha + \sum_{i=0}^m a_iu_i(x))$ $(\beta + \sum_{i=0}^m a_iv_i(x))$

### Pairing RHS:
Previously, $C$ = $\sum_{i=0}^m a_iw_i(X)$ + $h(x)t(x)$

We need to adapt for $\alpha \, and \, \beta$ $\sum_{i=0}^m a_i(\beta u_i(x) + \alpha v_i(x) +  w_i(x) )$ + $h(x)t(x)$  

Which equals: $\sum_{i=0}^m a_i\beta u_i(x) + a_i\alpha v_i(x) +  a_iw_i(x)$ + $h(x)t(x)$

In [8]:
from functools import reduce
import random

tau = GF(8)
# tau = GF(random.randint(1,curve_order-1))

def generate_powers_of_tau_G1(tau, degree):
    """
    tau is a random num between 1 and curve_order, degree is the highest d of the polynomial
    this generates the powers of tau, i.e. tau^0, tau^1, tau^2, tau^3, tau^4, tau^5
    """
    return [multiply(G1, int(tau ** i)) for i in range(degree + 1)]

def generate_powers_of_tau_G2(tau, degree):
    """
    same for G2
    """
    return [multiply(G2, int(tau ** i)) for i in range(degree + 1)]

def inner_product(ec_points, coeffs):
    """
    ec_points is the powers of tao, i.e. [G1, tau*G1, tau^2*G1, tau^3*G1, ...]
    coeffs are the coefficients of the polynomial after lagrange
    this computes the inner product of the 2 vectors
    i.e. Ux[0] * tau^0 + Ux[1] + tau^1..
    e.g. 123*tau^0*G1 + 456*tau^1*G1 + 789*tau^2*G1 + 123*tau^3*G1
    e.g. multiply(G1, 123*tau^0) pointadd multiply(G1, 456*tau^1) pointadd multiply(G1, 789*tau^2) 
    the result is 1 point on the curve representing the whole polynomial
    Z1 is the identity element
    """
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

# evaluate then convert
powers_of_tau_G1 = generate_powers_of_tau_G1(tau, 2 * Tx.degree)
powers_of_tau_G2 = generate_powers_of_tau_G2(tau, 2 * Tx.degree)
# print(powers_of_tau_G1)

# Trusted Setup
alpha1 = multiply(G1, 2)
beta2 = multiply(G2, 3)



# Prover generating AB
A_1_old = inner_product(powers_of_tau_G1, Ux.coeffs[::-1])
B_2_old = inner_product(powers_of_tau_G2, Vx.coeffs[::-1])


A_1_new = add(A_1_old, alpha1)
B_2_new = add(B_2_old, beta2)
lhs = pairing(B_2_new,A_1_new)

# Aold Bold = C
# A = Aold + alpha, B = Bold + beta
# AB = alpha beta + C
# AB = alpha beta + [alpha b + beta a] + AoldBold
# C needs to be a G1 point


alpha = 2
beta = 3

# Trusted Setup for C
Ux_prime = alpha * Ux
Vx_prime = beta * Vx
trusted_setup_beta_G1 = inner_product(powers_of_tau_G1, Ux_prime.coeffs[::-1])
trusted_setup_alpha_G1 = inner_product(powers_of_tau_G1, Vx_prime.coeffs[::-1])


# print(C_1prime)
# print(Wx)


# A_1new = add(A_1, alpha1)
# B_2new = add(B_2, beta2)
# lhs = pairing(B_2new, A_1new)

# rhs = pairing(G2,C_1prime)

# if(eq(lhs,rhs)):
#     print("true")
# else:
#     print("false")








# # Introducing alpha and beta to either side of pairing equation
# # 2 and 3 are arbitrary numbers chosen
# alpha1 = multiply(G1, 2)
# beta2 = multiply(G2, 3)

# A_1new = add(A_1, alpha1)
# B_2new = add(B_2, beta2)

# lhs = pairing(B_2new, A_1new)

# # RHS
# rhs_1 = pairing(beta2, alpha1)
# rhs_2 = pairing(beta2,A_1)
# rhs_3 = pairing(B_2, alpha1)
# rhs_4 = pairing(G2, C_1)
# rhs = rhs_1 * rhs_2 * rhs_3 * rhs_4

# if(eq(lhs,rhs)):
#     print("true")
# else:
#     print("false")

(7231593109241136996379804975294316287077409790236125900914040946802458790156, 2243199413637058191742357464895766628691785712428269937786752457704964437849)
((6687769990805907354780494181282853495775531704669382891331850594658479351500, 19116992951627598582317588856198769755469416288530602659351917964741210768765), (15787594042206970808240085135298324499426108722248997381539398767980414201574, 19814467509690106414989851308094880700571204308097873205060921740317127423242))
